In [11]:
# 최종 정확도 확인
netname = './nets/ex04_net_fin.pkl'
model = torch.load(netname)
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 78.68999481201172


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler

batch_size=16
learning_rate=0.002
num_epoch=100

In [3]:
#cifar_train=dset.CIFAR10("CIFAR10/", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)

# (2) Data augmentation
'''
cifar_train=dset.CIFAR10("CIFAR10/",train=True,
                         transform=transforms.Compose([
                            transforms.Scale(36),
                            transforms.CenterCrop(32),
                            transforms.RandomHorizontalFlip(),
                            transforms.Lambda(lambda x: x.rotate(90)),
                            transforms.ToTensor()                              
                         ]))
'''
# (4) Data Normalization

cifar_train=dset.CIFAR10("CIFAR10/", train=True,
                         transform=transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
                         ])
                         , target_transform=None, download=True)
cifar_test=dset.CIFAR10("CIFAR10/", train=False,
                         transform=transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
                         ])
                         , target_transform=None, download=True)

#cifar_test=dset.CIFAR10("CIFAR10/", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)


Extracting CIFAR10/cifar-10-python.tar.gz to CIFAR10/
Files already downloaded and verified


In [4]:
def ComputeAccr(dloader, imodel):
  correct = 0
  total = 0

  for j, [imgs, labels] in enumerate(dloader):  # batch_size 만큼
    img = Variable(imgs, volatile=True).cuda()
    #label = Variable(labels) # y
    label = Variable(labels).cuda()

    output = imodel.forward(img)  # forward prop.
    _, output_index = torch.max(output, 1)

    total += label.size(0)
    correct += (output_index == label).sum().float()
  print("Accuracy of Test Data: {}".format(100*correct/total))

In [5]:
# === 3. 데이터 로드 함수 ===
train_loader=torch.utils.data.DataLoader(list(cifar_train)[:], batch_size=batch_size, shuffle=True, num_workers=2,drop_last=True)
test_loader=torch.utils.data.DataLoader(cifar_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

# === 4. 모델 선언 ===
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer=nn.Sequential(
        nn.Conv2d(3,16,3,padding=1),
        nn.BatchNorm2d(16),  # (5) Batch normalization
        nn.ELU(),
        nn.Dropout2d(0.2),  # (2) drop out
        nn.Conv2d(16,32,3,padding=1),
        nn.BatchNorm2d(32),
        nn.ELU(),
        nn.Dropout2d(0.2),
        nn.MaxPool2d(2,2),
        nn.Conv2d(32,64,3,padding=1),
        nn.BatchNorm2d(64),
        nn.ELU(),
        nn.Dropout(0.2),
        nn.MaxPool2d(2,2)
    )
    self.fc_layer=nn.Sequential(
        nn.Linear(64*8*8, 100),
        nn.BatchNorm1d(100),
        nn.ELU(),
        nn.Dropout(0.2),
        nn.Linear(100,50),
        nn.Linear(50,10)
    )

    # (3) weight initialization
    '''
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        init.kaiming_normal(m.weight.data)  # REUL 일 때
        m.bias.data.fill_(0)
      if isinstance(m, nn.Linear):
        init.kaiming_normal(m.weight.data)
        m.bias.data.fill_(0)
    '''
  
  def forward(self,x):
    out=self.layer(x)
    out=out.view(batch_size,-1)
    out=self.fc_layer(out)

    return out
model=CNN().cuda()

In [7]:
# === 5. loss, optimizer ===
loss_func=nn.CrossEntropyLoss()
#optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)  # (6) Adam optimizer

#scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.2)  # (7) learning rate decay
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.15)
model.train() # train mode 로 바꾸기 -> dropout, batch normalization에 영향을 줌.
# === 6. 학습 ===
for i in range(num_epoch):
  for j, [image, label] in enumerate(train_loader):
    x=Variable(image).cuda()
    y_=Variable(label).cuda()

    optimizer.zero_grad()
    output=model.forward(x)
    loss=loss_func(output, y_)
    loss.backward()
    optimizer.step()

    if j%1000==0:
      print(j,loss)

0 tensor(2.3200, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(1.9484, device='cuda:0', grad_fn=<NllLossBackward>)
2000 tensor(1.1832, device='cuda:0', grad_fn=<NllLossBackward>)
3000 tensor(0.7676, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(1.3953, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.7511, device='cuda:0', grad_fn=<NllLossBackward>)
2000 tensor(0.7835, device='cuda:0', grad_fn=<NllLossBackward>)
3000 tensor(1.5506, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.9438, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(1.7865, device='cuda:0', grad_fn=<NllLossBackward>)
2000 tensor(0.9521, device='cuda:0', grad_fn=<NllLossBackward>)
3000 tensor(0.7679, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(1.1676, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(1.2599, device='cuda:0', grad_fn=<NllLossBackward>)
2000 tensor(1.0281, device='cuda:0', grad_fn=<NllLossBackward>)
3000 tensor(0.7838, device='cuda:0', grad_fn=<NllLos

# **test 16**

batch_size=16

learning_rate=0.002

num_epoch=100

***Data Normalization***

***batch normalization***

***nn.ELU()***

***Drop out***

***nn.Linear(100,50),  nn.Linear(50,10)***

***adam optimizer***

***learning rate decay***

***ExponentialLR - gamma=0.15***

In [8]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 78.68999481201172


In [9]:
# 학습된 파라미터 저장
netname = './nets/ex04_t16_net.pkl'
torch.save(model, netname,)

In [10]:
# 학습된 파라미터 저장
netname = './nets/ex04_net_fin.pkl'
torch.save(model, netname,)

# **test 15**

batch_size=16

learning_rate=0.002

num_epoch=50

***Data Normalization***

***batch normalization***

***nn.ELU()***

***Drop out***

***nn.Linear(100,50),  nn.Linear(50,10)***

***adam optimizer***

***learning rate decay***

***ExponentialLR - gamma=0.15***

In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 77.04000091552734


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t15_net.pkl'
torch.save(model, netname,)

# **test 14**

batch_size=16

learning_rate=0.002

num_epoch=80

nn.Linear(100,50),  nn.Linear(50,10)

***Data Normalization***

***nn.ELU()***

***Drop out***

***batch normalization***

***adam optimizer***

***learning rate decay***

***ExponentialLR - gamma=0.15***

In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 76.97999572753906


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t14_net.pkl'
torch.save(model, netname,)

# **test 13**

batch_size=16

learning_rate=0.002

num_epoch=100

Drop out

***Data Normalization***

***nn.ELU()***

***batch normalization***

***adam optimizer***

***learning rate decay***

***ExponentialLR - gamma=0.15***

In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 77.58999633789062


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t13_net.pkl'
torch.save(model, netname,)

# **test 12**

batch_size=16

learning_rate=0.002

num_epoch=50

***Data Normalization***

***nn.ELU()***

***batch normalization***

***adam optimizer***

***learning rate decay***

***ExponentialLR - gamma=0.2***

In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 75.04000091552734


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t12_net.pkl'
torch.save(model, netname,)

# **test 11**

batch_size=16

learning_rate=0.002

num_epoch=50

weight initialization

***Data Normalization***

***nn.ELU()***

***batch normalization***

***adam optimizer***

***learning rate decay***

***ExponentialLR - gamma=0.2***

In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 75.16999816894531


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t11_net.pkl'
torch.save(model, netname,)

# **test 10**

batch_size=16

learning_rate=0.002

num_epoch=30

StepLR -> ExponentialLR

***Data Normalization***

***nn.ELU()***

***batch normalization***

***adam optimizer***

***learning rate decay***

In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 74.98999786376953


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t10_net.pkl'
torch.save(model, netname,)

# **test 9**

batch_size=20

learning_rate=0.002

num_epoch=300

Data Normalization

***nn.ELU()***

***batch normalization***

***adam optimizer***

***learning rate decay***

In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 75.6199951171875


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t9_net.pkl'
torch.save(model, netname,)

# **test 8**

batch_size=20

learning_rate=0.015

num_epoch=200

***nn.ELU()***

***batch normalization***

***adam optimizer***

***learning rate decay***

In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 73.77999877929688


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t8_net.pkl'
torch.save(model, netname,)

# **test 7**

num_epoch=300

nn.Linear(100,50), nn.Linear(50,10)

***nn.ELU()***

***batch normalization***

***adam optimizer***

***learning rate decay***

In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 74.77999877929688


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t7_net.pkl'
torch.save(model, netname,)

# **test 6**

num_epoch=100

adam optimizer

***nn.ELU()***

***batch normalization***

***learning rate decay***


In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 74.81999969482422


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t6_net.pkl'
torch.save(model, netname,)

# **test 5**

num_epoch=100

learning rate decay

***nn.ELU()***

***batch normalization***



In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 70.81999969482422


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t5_net.pkl'
torch.save(model, netname,)

# **test 4**

***nn.ELU()***

***batch normalization***

adam optimizer


In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 68.19999694824219


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t4_net.pkl'
torch.save(model, netname,)

# **test 3**

***nn.ELU()***

batch normalization

In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 63.269996643066406


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t3_net.pkl'
torch.save(model, netname,)

# **test 2**

nn.ReLU() -> nn.LeakyReLU()

In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 17.299999237060547


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t2_net.pkl'
torch.save(model, netname,)

# **test 1**

nn.ReLU() -> nn.ELU()

In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 30.09000015258789


In [ ]:
# 학습된 파라미터 저장
netname = './nets/ex04_t1_net.pkl'
torch.save(model, netname,)